In [ ]:
from pycocotools.coco import COCO
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
from random import randint
from collections import defaultdict

In [ ]:
coco_folder = '/mnt/DATASSD/DataSandbox/coco'
dp_coco = COCO( coco_folder + '/annotations/custom_ann_train2014_v06.json')


In [ ]:
# Get img id's for the dataset.
im_ids = dp_coco.getImgIds()
# Select a random image id.
Selected_im = im_ids[randint(0, len(im_ids))] # Choose im no 57 to replicate 
# print(Selected_im)
# Selected_im = 262145
# Load the image
im = dp_coco.loadImgs(Selected_im)[0]  
# Load Anns for the selected image.
ann_ids = dp_coco.getAnnIds( imgIds=im['id'] )
anns = dp_coco.loadAnns(ann_ids)
# Now read and b
im_name = os.path.join( coco_folder + '/train2014', im['file_name'] )

In [ ]:
print(im_name)
I=cv2.imread(im_name)
plt.imshow(I[:,:,::-1]); plt.axis('off'); plt.show()

In [ ]:
ids = dp_coco.getAnnIds(imgIds=[Selected_im])
print(ids)
target = dp_coco.loadAnns(ids)
print(len(target))
file_name = dp_coco.loadImgs(Selected_im)[0]['file_name']
print(file_name)
mask = dp_coco.annToMask(target[0])
print(mask.shape, mask.max())


In [ ]:
def GetDensePoseMask(Polys):
    MaskGen = np.zeros([256,256])
    for i in range(1,15):
        if(Polys[i-1]):
            current_mask = mask_util.decode(Polys[i-1])
            MaskGen[current_mask>0] = i
    return MaskGen

In [ ]:
points0 = anns[0]['segmentation'][0]
pointsU = anns[0]['dp_U']
pointsV = anns[0]['dp_V']
pointsx = anns[0]['dp_x']
pointsy = anns[0]['dp_y']

In [ ]:
for i in range(0,len(points0),2):
    x = int(points0[i])
    y = int(points0[i+1])
    I = cv2.circle(I, (x,y), 2,(255,255,0),2)
    
I_vis=I.copy()/2 # Dim the image.
I_vis2=I.copy()

for ann in anns:  
    bbr =  np.array(ann['bbox']).astype(int) # the box.
    if( 'dp_masks' in ann.keys()): # If we have densepose annotation for this ann, 
        Mask = GetDensePoseMask(ann['dp_masks'])
        mascaras = ann['dp_masks']
        ################
        x1,y1,x2,y2 = bbr[0],bbr[1],bbr[0]+bbr[2],bbr[1]+bbr[3]
        print(x1,y1,x2,y2)
        I_vis2 = cv2.rectangle(I_vis2, (x1,y1), (x2,y2),(255,0,0),2)
        x2 = min( [ x2,I.shape[1] ] );  y2 = min( [ y2,I.shape[0] ] )
        ################ 
        print(int(x2-x1),int(y2-y1))
        MaskIm = cv2.resize( Mask, (int(x2-x1),int(y2-y1)) ,interpolation=cv2.INTER_NEAREST)

        MaskBool = np.tile((MaskIm==0)[:,:,np.newaxis],[1,1,3])
        #  Replace the visualized mask image with I_vis.
        Mask_vis = cv2.applyColorMap( (MaskIm*15).astype(np.uint8) , cv2.COLORMAP_PARULA)[:,:,:]
        Mask_vis[MaskBool]=I_vis[y1:y2,x1:x2,:][MaskBool]
        I_vis[y1:y2,x1:x2,:] = I_vis[y1:y2,x1:x2,:]*0.3 + Mask_vis*0.7



'''for i in range(0,len(points1),2):
    x = int(points1[i])
    y = int(points1[i+1])
    I = cv2.circle(I, (x,y), 2,(255,0,255),2)'''

print(I_vis.shape)
fig = plt.figure(figsize=[15,30])
plt.subplot(4,2,1)
plt.imshow(I[:,:,::-1])

plt.subplot(4,2,2)
plt.imshow(I_vis)

plt.subplot(4,2,3)
plt.imshow(MaskIm)

plt.subplot(4,2,4)
plt.imshow(Mask_vis)

plt.subplot(4,2,5)
plt.imshow(I2[:,:,::-1])

plt.subplot(4,2,6)
plt.imshow(mask)

plt.subplot(4,2,7)
plt.imshow(Mask)

plt.subplot(4,2,8)
plt.imshow(I_vis2)

plt.show()